## Relax Inc. Challenge - Predict User Adoption

Two tables - one with user info the other with user engagement. I think I'll need to aggregate the engagement data to flag users as having adopted the software. This needs to based on the logic of logging in on 3 seperate days in any 7-day period.

Then we can predict using the user data.

## 0.5 Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

## 1.0 Load and Prep Data

### User Data

In [2]:
users = pd.read_csv('../data/takehome_users.csv', encoding='latin-1')

users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
#check info
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [4]:
#check for total duplicates
users[users.duplicated(keep=False)]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id


In [5]:
#check for duplicate users
users['object_id'].value_counts(0)

1        1
8004     1
7995     1
7996     1
7997     1
        ..
4003     1
4004     1
4005     1
4006     1
12000    1
Name: object_id, Length: 12000, dtype: int64

In [6]:
#make sure T/F columns only have 1s and 0s
print(users['opted_in_to_mailing_list'].value_counts())

print(users['enabled_for_marketing_drip'].value_counts())

0    9006
1    2994
Name: opted_in_to_mailing_list, dtype: int64
0    10208
1     1792
Name: enabled_for_marketing_drip, dtype: int64


In [7]:
#check creation source for consistency
users['creation_source'].value_counts()

ORG_INVITE            4254
GUEST_INVITE          2163
PERSONAL_PROJECTS     2111
SIGNUP                2087
SIGNUP_GOOGLE_AUTH    1385
Name: creation_source, dtype: int64

With those checks passing, I just want to update and explore my date fields a bit.

In [8]:
users['creation_time'] = pd.to_datetime(users['creation_time'])

In [9]:
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'], unit='s')

In [10]:
users.dtypes

object_id                              int64
creation_time                 datetime64[ns]
name                                  object
email                                 object
creation_source                       object
last_session_creation_time    datetime64[ns]
opted_in_to_mailing_list               int64
enabled_for_marketing_drip             int64
org_id                                 int64
invited_by_user_id                   float64
dtype: object

In [11]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0


In [12]:
#check min an max dates
print(users['creation_time'].min(), users['creation_time'].max())
print('--'*50)
print(users['last_session_creation_time'].min(), users['last_session_creation_time'].max())

2012-05-31 00:43:27 2014-05-30 23:59:19
----------------------------------------------------------------------------------------------------
2012-05-31 08:20:06 2014-06-06 14:58:50


### Engagement Data

In [13]:
engage = pd.read_csv('../data/takehome_user_engagement.csv')

engage.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [14]:
engage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [15]:
#is the last column all 1s
engage['visited'].value_counts()

1    207917
Name: visited, dtype: int64

In [16]:
#convert timestamp to datetime
engage['time_stamp'] = pd.to_datetime(engage['time_stamp'])

engage.dtypes

time_stamp    datetime64[ns]
user_id                int64
visited                int64
dtype: object

## 2.0 Adding & Exploring Features

### Engagement Data
We'll snake draft it and start with engagement this time.

In [24]:
# Convert 'time_stamp' column to datetime type
def count_engagement(df):
    """Run through engagement data and count the number of visits in 7-day windows """
    count_df = df.copy()

    count_df['time_stamp'] = pd.to_datetime(count_df['time_stamp'])

    # Sort the DataFrame by 'time_stamp'
    count_df = count_df.sort_values('time_stamp')

    # Set 'time_stamp' column as the DataFrame index
    count_df = count_df.set_index('time_stamp')

    # Resample the DataFrame using a 7-day window and count the number of 'visited' per user
    count_per_user = count_df.groupby('user_id').apply(lambda x: x.resample('1D')['visited'].max().rolling(7, min_periods=1).sum()) #using max so we don't double count a single day. Want max 1, if anything. 

    output = count_per_user.reset_index().dropna() #dropping blank dates

    return output

In [25]:
#create new datframe with 7-day counts
engage_count = count_engagement(engage)

In [28]:
engage_count.head()

,user_id,time_stamp,visited
0,1,2014-04-22,1.0
1,2,2013-11-15,1.0
2,2,2013-11-16,1.0
3,2,2013-11-17,1.0
4,2,2013-11-18,1.0


In [34]:
#create final dataframe with max of 7-day logins
user_max = engage_count.groupby('user_id')['visited'].max().reset_index().rename(columns={'visited': 'max_7D_logins'})

user_max.head()

,user_id,max_7D_logins
0,1,1.0
1,2,3.0
2,3,1.0
3,4,1.0
4,5,1.0


In [33]:
#validate we shouldn't see more than 7
user_max['max_7D_logins'].max()

7.0

In [35]:
#nice! Let's create a flag for our target features
user_max['is_engaged'] = user_max['max_7D_logins'] >= 3.0

user_max.head()

,user_id,max_7D_logins,is_engaged
0,1,1.0,False
1,2,3.0,True
2,3,1.0,False
3,4,1.0,False
4,5,1.0,False
